Задание 3: Проверить правильность выполнения задания 1 путем запуска в цикле 1000 разных матриц чтобы убедиться что у вас не получилось случайно ни в одной из них лишних доминируемых/доминирующих стратегий (должно быть ровно K)

## Задание 1

Написать функцию генерации платежной матрицы игры заданной размерности M x N. При этом добавить опциональные параметры (в Матлабе - написать несколько разных функций), позволяющих:
* Сделать просто произвольную платежную матрицу из целых чисел M x N
* Создать такую платежную матрицу M x N где есть ровно k<N последовательно доминирующих стратегий (т.е. если удалить доминирующую строчку, то в оставшейся матрице все равно останется доминирующая строчка)
* Создать такую платежную матрицу M x N где есть ровно k<N последовательно доминируемых стратегий (т.е. по методу СДС должно удаляться ровно k строк)


In [2]:
import numpy as np
import random

random.seed(42)

In [52]:
def sort_first_k_rows_items(matrix: np.ndarray, rows_sort_count: int, reverse: bool = False):
    k = 0
    while k < rows_sort_count and k < matrix.shape[0]:
        # Для каждого столбца
        for j in range(matrix.shape[1]):
            max_by_column_row_num = k
            previous_max = matrix[k][j]

            # Находим максимум в столбце, начиная с k-ой строки
            for i in range(k, matrix.shape[0]):
                if reverse:
                    if matrix[i][j] < previous_max:
                        max_by_column_row_num = i
                        previous_max = matrix[i][j]
                else:
                    if matrix[i][j] > previous_max:
                        max_by_column_row_num = i
                        previous_max = matrix[i][j]

            # Меняем местами элемент k-ой строки и максимальный элемент в этом столбце, начиная с k-ой строки
            matrix[max_by_column_row_num][j] = matrix[k][j]
            matrix[k][j] = previous_max
        k += 1

def generate_cost_matrix(rows_count: int, columns_count: int, dominant_strategies_count: int = 0,
                        dominated_strategies_count: int = 0) -> np.ndarray:
    result_cost_matrix = np.zeros((rows_count, columns_count))
    
    # заполнили рандомно платежную матрицу
    for i in range(result_cost_matrix.shape[0]):
        for j in range(result_cost_matrix.shape[1]):
            result_cost_matrix[i][j] = random.randint(1, 500)
    
    # отсортируем первые k строк так, чтобы они были доминирующими стратегиями
    sort_first_k_rows_items(result_cost_matrix, dominant_strategies_count)

    # отсортируем первые k строк так, чтобы они были доминируемыми стратегиями
    sort_first_k_rows_items(result_cost_matrix, dominated_strategies_count, reverse = True)
    return result_cost_matrix    

## Задание 2

Реализовать метод СДС для произвольных платежных матриц (функция должна выводить, сколько раз у нее получилось удалить строчку доминируемой стратегии)